In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import copy

from shapely import Point
from shapely import affinity, set_precision
from shapely.plotting import plot_points

#sys.path.insert(1, '/Users/neoh/Documents/git/eeroqlab/fabrication/')
sys.path.insert(1, '/Users/helium/Documents/git/eeroqlab/fabrication')
from zeroheliumkit.settings import *
from zeroheliumkit.helpers.plotting import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

GRID_SIZE = 1e-4